<a href="https://colab.research.google.com/github/toshif/colab1/blob/main/simple_haiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/deepmind/dm-haiku

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-i2gv6fsh
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-i2gv6fsh
  Created wheel for dm-haiku: filename=dm_haiku-0.0.6.dev0-py3-none-any.whl size=537308 sha256=158e4417ad68bac4ed63ba5cb8c2587c89bdbdfe400df3a8bd6da64ca47fea9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wnlfkk3e/wheels/06/28/69/ebaac5b2435641427299f29d88d005fb4e2627f4a108f0bdbc
Successfully built dm-haiku


In [ ]:
!pip install optax

     |████████████████████████████████| 118 kB 4.2 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 


In [ ]:
import functools
import math
from typing import Tuple, TypeVar
import warnings

import haiku as hk
import jax
import jax.numpy as jnp
import optax
import numpy as np
import pandas as pd
import plotnine as gg

T = TypeVar('T')
Pair = Tuple[T, T]

gg.theme_set(gg.theme_bw())
warnings.filterwarnings('ignore')

In [ ]:
x = jnp.ones([3, 2, 4])
y = jnp.ones([3, 8]) * 5

def my_net_fn(x):
  mlp = hk.Sequential([
      hk.Flatten(),
  ])
  return mlp(x)

model = hk.transform(my_net_fn)

rng = jax.random.PRNGKey(428)
params = model.init(rng, x)

model.apply(params, None, x)

DeviceArray([[1., 1., 1., 1., 1., 1., 1., 1.],
             [1., 1., 1., 1., 1., 1., 1., 1.],
             [1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

In [ ]:
@jax.jit
def loss(params, x, y):
  print(f'params={params}, x={x.shape}, y={y.shape}')
  pred, _ = model.apply(params, None, x)
  return jnp.mean(jnp.square(pred - y))

@jax.jit
def update(step, params, opt_state, x, y):
  l, grads = jax.value_and_grad(loss)(params, x, y)
  grads, opt_state = opt.update(grads, opt_state)
  params = optax.apply_updates(params, grads)
  return l, params, opt_state

opt = optax.adam(1e-3)
opt_state = opt.init(params)

loss1 = loss(params, x, y)
print(f'loss1={loss1}')

for step in range(10):
  if step % 100 == 0:
    x, y = next(valid_ds)
    print("Step {}: valid loss {}".format(step, loss(params, x, y)))

  x, y = next(train_ds)
  train_loss, params, opt_state = update(step, params, opt_state, x, y)
  if step % 100 == 0:
    print("Step {}: train loss {}".format(step, train_loss))